In [1]:
import numpy as np
import random
import time

params = [[150, 0.95], [200, 0.95], [300, 0.85], [400, 0.6], [500, 0.6], [1000, 0.4], [1500, 0.4]]

ОБЩИЕ ФУНКЦИИ

In [2]:
def greedy_search(degrees, graph): #Жадный поиск
    candidates = list(degrees.keys())
    clique = [candidates[0]]

    while True:
        candidate_index = 0 
        to_drop = []

        for candidate in candidates: #Перестроение списка кандидатов
            for vertex in clique:
                if graph[vertex][candidate] == 0 or vertex == candidate:
                    to_drop.append(candidate_index)
                    break
                
            candidate_index += 1

        candidates = [i for j, i in enumerate(candidates) if j not in to_drop]
        
        if len(candidates) == 0: #Если кандидаты ещё есть - добавление в клику лучшего, иначе завершение программы
            break
        clique.append(candidates[0])

    return clique

In [3]:
def calculate_degrees_in_sets(set1, set2, graph):
    degrees = {}
    for i in range(len(set1)):
        degree = 0
        for j in range(len(set2)):
            if set1[i] != set2[j] and graph[set1[i]][set2[j]] == 1.0:
                degree += 1

        degrees[set1[i]] = degree

    degrees = {k: v for k, v in sorted(degrees.items(), key=lambda item: item[1], reverse = True)}

    return degrees

In [67]:
def get_graph_lists(graph): # Преобразование графа в список и получение списков смежности и степеней для вершин
    graph = list(graph)
    for i in range(len(graph)):
        graph[i] = list(graph[i])

    adj_list = {}
    degrees = {}
    for i in range(len(graph)):
        temp = []
        degree = 0
        for j in range(len(graph)):
            if graph[i][j] == 1.0:
                degree += 1
                temp.append(j)
        adj_list[i] = temp
        degrees[i] = degree

    degrees = {k: v for k, v in sorted(degrees.items(), key=lambda item: item[1], reverse = True)}
    return graph, degrees, adj_list

ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ АЛГОРИТМОВ

In [5]:
def is_clique(graph, set):
    for i in range(len(set)):
        for j in range(i, len(set)):
            if graph[i][j] == 0:
                return False
            
    return True

In [6]:
def set_num_edges(set, graph):
    edges_num = 0
    vertex_num = len(set)
    
    for i in range(vertex_num):
        for j in range(i, vertex_num):
            if graph[set[i]][set[j]] > 0:
                edges_num += 1

    return edges_num

In [7]:
def rebuild_candidates(candidates, clique, graph):
    candidate_index = 0 
    to_drop = []

    #print(clique, candidates)

    for candidate in candidates: #Перестроение списка кандидатов
        for vertex in clique:
            #print(vertex, candidate)
            if vertex == candidate or graph[vertex][candidate] == 0:
                to_drop.append(candidate_index)
                break
            
        candidate_index += 1

    candidates = [i for j, i in enumerate(candidates) if j not in to_drop]
    return candidates

In [8]:
def find_best_quality(current, qualityes):
    best = current
    for i in range(len(qualityes)):
        if qualityes[i][2] > best:
            best = qualityes[i]

    return best

In [9]:
def legal_neighbourhood(clique, vertices, graph, taboo_list):
    qualityes = []

    for i in range(len(clique)):
        temp_clique = [k for k in clique if k != clique[i]] #clique.pop(i)
        temp_candidates = rebuild_candidates(vertices, temp_clique, graph)
        temp_num_candidates = len(temp_candidates)
        for j in range(temp_num_candidates):
            if taboo_list[temp_candidates[i]] == 0: 
                qualityes.append([i, j, len(rebuild_candidates(temp_candidates, temp_clique.append(temp_candidates[j], graph)))])
    
    return qualityes

In [10]:
def starting_solution(graph, degrees, adj_list, clique_size):
    vertices = list(adj_list.keys())
    clique = greedy_search(degrees, graph)
    not_clique = [i for i in vertices if i not in clique]
    while len(clique) < clique_size:
        clique.append(not_clique[random.randint(0, len(not_clique) - 1)])
    return clique

In [11]:
def k_opt_neighbourhood(graph, taboo_list, minInClique, maxOutClique, minIn, maxOut):
    T = []

    minInClique_list = list(minInClique.keys())
    maxOutClique_list = list(maxOutClique.keys())

    for i in range(len(minInClique)):
        for j in range(len(maxOutClique)):
            if taboo_list[minInClique_list[i]] == 0: 
                delta = int(graph[minInClique_list[i]][maxOutClique_list[j]])
                if delta == 0:
                    T.append([minInClique_list[i], maxOutClique_list[j], maxOut - minIn - int(graph[minInClique_list[i]][maxOutClique_list[j]])])

    if len(T) > 0:
        return(T[random.randint(0, len(T) - 1)])
    else:
        if len(minInClique_list) < 1 or len(maxOutClique_list) < 1:
            print(maxOut, maxOutClique_list, minIn, minInClique_list)
        if len(minInClique_list) == 1:
            u = minInClique_list[0]
        else:
            u = minInClique_list[random.randint(0, len(minInClique_list) - 1)]

        if len(maxOutClique_list) == 1:
            v = maxOutClique_list[0]
        else:
            v = maxOutClique_list[random.randint(0, len(maxOutClique_list) - 1)]
        return([u, v, maxOut - minIn])

АЛГОРИТМЫ

AMTS

In [12]:
def TS_AMTS(graph, adj_list, clique, clique_size, search_depth, iterations, movements, graph_params):
    goal_number = int(clique_size * (clique_size - 1) / 2)
    num_vertices = len(graph)
    It = 0
    best_clique = clique
    best_num_edges = set_num_edges(clique, graph)
    taboo_list = [0] * num_vertices
    vertices = list(adj_list.keys())
    num_edges = best_num_edges
    quality = []
    v, u = 0, 0

    while It < search_depth:
        #print("Taboo", clique, num_edges)

        not_clique = [i for i in vertices if i not in clique]
        local_degrees = calculate_degrees_in_sets(clique, clique, graph)
        outer_degrees = calculate_degrees_in_sets(not_clique, clique, graph)

        minIn = list(local_degrees.values())[-1]
        #print("Min In", minIn, local_degrees)
        maxOut = list(outer_degrees.values())[0]
        #print("Max Out", maxOut, outer_degrees)

        minInClique = {k: v for k, v in sorted(local_degrees.items(), key=lambda item: item[1], reverse = True) if v == minIn and taboo_list[k] == 0}
        maxOutClique = {k: v for k, v in sorted(outer_degrees.items(), key=lambda item: item[1], reverse = True) if v == maxOut and taboo_list[k] == 0}
        #minInClique = [k for k, v in local_degrees if (v == minIn and k not in taboo_list)] 
        #maxOutClique = [k for k, v in outer_degrees if (v == maxOut and k not in taboo_list)] 
        #print("Taboo list", taboo_list)
        if len(minInClique) == 0:
            minInClique = {k: v for k, v in sorted(local_degrees.items(), key=lambda item: item[1], reverse = True) if v == minIn}    
        if len(maxOutClique) == 0:
            maxOutClique = {k: v for k, v in sorted(outer_degrees.items(), key=lambda item: item[1], reverse = True) if v == maxOut}

        quality = k_opt_neighbourhood(graph, taboo_list, minInClique, maxOutClique, minIn, maxOut)
        
        if quality[2] != num_edges:
            new_clique = [k for k in clique if k != quality[0]] #clique.pop(clique.index(quality[0]))
            new_clique.append(quality[1])
            new_num_edges = quality[2]
        else:
            P = min([(goal_number - quality[2] + 2) / len(vertices), 0.1])
            if random.random() > P:

                u = quality[0]
                v = quality[1]
                new_clique = [k for k in clique if k != u] #clique.pop(u)

                new_clique.append(v)
                new_num_edges = quality[2]
            else:
                u = clique[random.randint(0, len(clique) - 1)]
                worseInClique = {k : v for k, v in outer_degrees.items() if v < clique_size * graph_params[2]}
                
                if len(worseInClique) == 0:
                    mulitiplyer = 2
                    while len(worseInClique) == 0:
                        worseInClique = {k : v for k, v in outer_degrees.items() if v < clique_size * graph_params[2] * mulitiplyer}
                        mulitiplyer += 1

                if len(worseInClique) > 1:
                    v = not_clique[list(worseInClique.values())[random.randint(0, len(worseInClique) - 1)]]
                else:
                    #print(worseInClique)
                    v = list(worseInClique.values())[0]
                new_clique = [k for k in clique if k != u] #clique.pop(u)

                new_clique.append(v)
                new_num_edges = set_num_edges(clique, graph)
        
        #print(clique, new_clique)

        movements[v] += 1
        movements[u] += 1

        IsOk = True
        for counter in range(len(movements)):
            if movements[counter] > clique_size:
                IsOk = False
            else:
                IsOk = True
                break
        if IsOk == False:
            movements = [0] * len(graph)

        #if len(clique) > clique_size:
            #print("ACHTUNG",clique_size, clique, num_edges, goal_number)
            #pass

        if new_num_edges == goal_number:
            #print("Taboo complete", new_clique, new_num_edges, "iterations", iterations, It)
            return new_clique, movements

        clique = new_clique
        num_edges = new_num_edges
        
        iterations += 1

        if new_num_edges > best_num_edges:
            best_clique = new_clique
            best_num_edges = new_num_edges
            It = 0
        else:
            It += 1 

        for i in range(num_vertices):
            if taboo_list[i] > 0:
                taboo_list[i] -= 1
        l1 = clique_size - num_edges
        l = min([l1, 10])
        C = max(clique_size / 40, 6)
        taboo_list[u] = int(l + random.randint(0, int(C)))
        taboo_list[v] = int(0.6 * l + random.randint(0, int(0.6 * C))) + 1

        #print(clique)

    #print("Taboo best", best_clique, best_num_edges)
    return best_clique, movements

In [13]:
def AMTS(degrees, graph, adj_list, clique_size, search_depth, max_iters, graph_params):
    clique = starting_solution(graph, degrees, adj_list, clique_size)
    iterations = 0
    movements = [0] * len(graph)
    vertices = [i for i in range(len(graph))]

    while iterations < max_iters:
        clique, movements = TS_AMTS(graph, adj_list, clique, clique_size, search_depth, iterations, movements, graph_params)
        #print("AMTS", clique, set_num_edges(clique, graph))
        if is_clique(graph, clique):
            return clique
        if set_num_edges(clique, graph) == int(clique_size * (clique_size - 1) / 2):
            return clique
        else:
            
            minMovementsVal = min(movements)
            minMovementsInd = [i for i in range(len(graph)) if movements[i] == minMovementsVal]
            clique = [minMovementsInd[random.randint(0, len(minMovementsInd) - 1)]]
            while len(clique) < clique_size:
                not_clique = [i for i in vertices if i not in clique]
                restart_candidates = calculate_degrees_in_sets(not_clique, clique, graph)
                max_restart_degree = max(list(restart_candidates.values()))
                mxdeg_restart_candidates = {k : movements[k] for k in restart_candidates.keys() if restart_candidates[k] == max_restart_degree}
                #print(clique, clique_size, mxdeg_restart_candidates)
                if len(mxdeg_restart_candidates) == 1:
                    v = list(mxdeg_restart_candidates.keys())[0]
                else:
                    min_movements = min(list(mxdeg_restart_candidates.values()))
                    final_candidates = [i for i in list(mxdeg_restart_candidates.keys()) if mxdeg_restart_candidates[i] == min_movements]
                    #print(final_candidates)
                    if len(final_candidates) == 1:
                        v = final_candidates[0]
                    else:
                        v = final_candidates[random.randint(0, len(final_candidates) - 1)]
                clique.append(v)

        #print("AMTS", clique, set_num_edges(clique, graph))

        iterations += 1

    return None

In [14]:
def test_AMTS(graph, graph_params, degrees, adj_list, max_iterations):
    start_time = time.time()
    end_time = start_time + 50
    clique = []
    for i in range(int(graph_params[-2]), int(graph_params[-1])):
        sampe = AMTS(degrees, graph, adj_list, i, 50, max_iterations, graph_params)
        #AMTS(degrees, graph, adj_list, clique_size, search_depth, max_iters, graph_params)
        #print(sampe)
        if sampe == None:
            i -= 1
            break
        else:
            clique = sampe
            end_time = time.time()

    return len(clique), end_time - start_time

DLS

In [15]:
def sets_intersection(set1, set2):
    common = []
    for vertex in set1:
        if vertex in set2:
            common.append(vertex)
    
    return common

In [16]:
def SelectMinPenalties(penalties, set):
    set_penalties = {}
    for i in range(len(set)):
        set_penalties[set[i]] = penalties[set[i]]
    
    min_set_penalties = {k: v for k, v in set_penalties.items() if v == min(list(set_penalties.values()))}
    #print(min_set_penalties, len(min_set_penalties), random.randint(0, len(min_set_penalties) - 1))
    return list(min_set_penalties.keys())[random.randint(0, len(min_set_penalties) - 1)]

In [17]:
def DropNonAdjacentToV(clique, vertex, graph):
    to_drop = []
    for i in range(len(clique)):
        if clique[i] != vertex:
            #print(clique, clique[i], vertex)
            if graph[vertex][clique[i]] == 0:
                to_drop.append(clique[i])
    clique = [i for i in clique if i not in to_drop]
    return clique

In [18]:
def rebuild_level_neighbourhood(clique, graph, vertices, used):
    temp_candidates = []
    new_candidates = []
    #print(len(clique), len(temp_candidates))
    for i in range(len(clique)):
        temp_clique = [k for k in clique if k != clique[i]] #clique.pop(i)
        #print("temp clique", temp_clique)
        if len(temp_clique) > 0:
            temp = rebuild_candidates(vertices, temp_clique, graph)
            temp_candidates.append([k for k in temp if k not in clique and k not in used])

    for i in range(len(temp_candidates)):
        #print(temp_candidates[i])
        for j in range(len(temp_candidates[i])):
            if temp_candidates[i][j] not in new_candidates:
                new_candidates.append(temp_candidates[i][j])
        
    return new_candidates

In [19]:
def DLS_expand(graph, clique, penalties, iterations, vertices):
    candidates = rebuild_candidates(vertices, clique, graph)
    v = None
    while len(candidates) > 0:
        v = SelectMinPenalties(penalties, candidates)
        clique.append(v)
        iterations += 1
        #print("expand", clique)
        #print(candidates, "\n")
        candidates = rebuild_candidates(vertices, clique, graph)
    
    return clique, v, iterations

In [20]:
def DLS_PlateauSearch(graph, clique, recorded_clique, vertices, penalties, iterations, max_iters):
    candidates = rebuild_candidates(vertices, clique, graph)
    intersection = sets_intersection(clique, recorded_clique)
    level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used = [])

    used = []
    v = None
    #print("plateau")
    while len(candidates) == 0 and len(level_candidates) != 0 and len(intersection) != 0:
        #if iterations >= max_iters:
            #break
        #print("level", level_candidates)
        v = SelectMinPenalties(penalties, level_candidates)
        #penalties[v] += 5
        #print(v)
        used.append(v)
        clique.append(v)
        clique = DropNonAdjacentToV(clique, v, graph)

        #print(clique)
        
        #print("plateau", clique)
        candidates = rebuild_candidates(vertices, clique, graph)
        intersection = sets_intersection(clique, recorded_clique)
        level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
        iterations += 1

    return clique, v, iterations

In [21]:
def DLS(graph, clique_size, penalty_delay, max_iters):
    vertices = [i for i in range(len(graph))]
    v = 0
    iterations = 0
    clique = [random.randint(0, len(graph) - 1)]
    penalties = [0] * len(graph)
    candidates = rebuild_candidates(vertices, clique, graph)

    while iterations < max_iters:
        clique, v, iterations = DLS_expand(graph, clique, penalties, iterations, vertices)

        if len(clique) == clique_size:
            return clique
        recorded_clique = clique
        clique, v, iterations = DLS_PlateauSearch(graph, clique, recorded_clique, vertices, penalties, iterations, max_iters)
        #print(clique, recorded_clique)
        candidates = rebuild_candidates(vertices, clique, graph)

        #print("DLS", clique)

        while len(candidates) > 0:
            #print("Were candidates")
            clique, v, iterations = DLS_expand(graph, clique, penalties, iterations, vertices)
            if len(clique) == clique_size:
                return clique
            clique, v, iterations = DLS_PlateauSearch(graph, clique, recorded_clique, vertices, penalties, iterations, max_iters)
            candidates = rebuild_candidates(vertices, clique, graph)
    
        for i in range(len(penalties)):
            if penalties[i] > 0:
                penalties[i] -= 1
            if v != None:
                penalties[v] = penalty_delay

        if penalty_delay > 1 and v != None:
            clique = [v]
        else:
            not_clique = [i for i in vertices if i not in clique]
            v = not_clique[random.randint(0, len(not_clique) - 1)]
            clique.append(v)
            clique = DropNonAdjacentToV(clique, v, graph)
    return None

In [22]:
def test_DLS(graph, graph_params, degrees, adj_list, max_iterations):
    start_time = time.time()
    end_time = start_time + 50
    clique = []
    for i in range(int(graph_params[-2]), int(graph_params[-1])):
        sampe = DLS(graph, i, 2, max_iterations)
        #AMTS(degrees, graph, adj_list, clique_size, search_depth, max_iters, graph_params)
        #print(sampe)
        if sampe == None:
            i -= 1
            break
        else:
            clique = sampe
            end_time = time.time()
            #print(clique, "\n")
            #for vertex in clique:
                #print(vertex, adj_list[vertex])
    return len(clique), end_time - start_time

PLS

In [23]:
def PLS_penalties(graph, clique, selections, max_selections, iterations, vertices, clique_size, penalties):
    used = []
    candidates = rebuild_candidates(vertices, [], graph)
    level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
    
    while iterations > 0 and selections < max_selections:
        while len(candidates) > 0 or len(level_candidates) > 0:
            candidates = rebuild_candidates(vertices, clique, graph)
            while len(candidates) > 0:
                #print(candidates)
                candidates = rebuild_candidates(vertices, clique, graph)
                candidate_penalties = {k : penalties[k] for k in candidates}
                #print(candidates, candidate_penalties)
                min_penalties = min(list(candidate_penalties.values()))
                min_pen_candidates = [i for i in candidates if penalties[i] == min_penalties]

                #print(candidates, min_pen_candidates)
                if len(min_pen_candidates) == 1:
                    v = min_pen_candidates[0]
                else:
                    v = min_pen_candidates[random.randint(0, len(min_pen_candidates) - 1)]
                clique.append(v)
                candidates = rebuild_candidates(vertices, clique, graph)

            if len(clique) == clique_size:
                return clique, selections#, penalties
            selections += 1
            #candidates = rebuild_candidates(vertices, [], graph)
            level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
            if len(level_candidates) > 0:
                candidate_penalties = {k : penalties[k] for k in level_candidates}
                min_penalties = min(list(candidate_penalties.values()))
                min_pen_candidates = [i for i in level_candidates if penalties[i] == min_penalties]

                #print(level_candidates, min_pen_candidates)
                if len(min_pen_candidates) == 1:
                    v = min_pen_candidates[0]
                else:
                    v = min_pen_candidates[random.randint(0, len(min_pen_candidates) - 1)]
                clique.append(v)
                clique = DropNonAdjacentToV(clique, v, graph)
                used.append(v)
        iterations -= 1
        for i in range(len(penalties)):
            if penalties[i] == 0:
                penalties[i] -= 1
            if i in used:
                penalties[i] += 1
        not_clique = [i for i in vertices if i not in clique]
        u = not_clique[random.randint(0, len(not_clique) - 1)]
        clique.append(u)
        clique = DropNonAdjacentToV(clique, u, graph)
    return clique, selections#, penalties

In [24]:
def PLS_degrees(graph, clique, degrees, selections, max_selections, iterations, vertices, clique_size, penalties):
    used = []
    candidates = rebuild_candidates(vertices, [], graph)
    level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)

    while iterations > 0 and selections < max_selections:
        while len(candidates) > 0 or len(level_candidates) > 0:
            candidates = rebuild_candidates(vertices, clique, graph)
            while len(candidates) > 0:
                candidates = rebuild_candidates(vertices, clique, graph)
                cand_degrees = {k : v for k, v in degrees.items() if k in candidates}
                #print(candidates, cand_degrees)
                max_deg = max(list(cand_degrees.values()))
                max_deg_candidates = [i for i in list(cand_degrees.keys()) if cand_degrees[i] == max_deg]

                if len(max_deg_candidates) == 1:
                    v = max_deg_candidates[0]
                else:
                    v = max_deg_candidates[random.randint(0, len(max_deg_candidates) - 1)]
                clique.append(v)
                #used.append(v)
                #print("expansion", clique, level_candidates, max_deg_candidates)
                candidates = rebuild_candidates(vertices, clique, graph)
                #print(candidates)

            if len(clique) == clique_size:
                return clique, selections#, penalties
            selections += 1
                #candidates = rebuild_candidates(vertices, [], graph)
            level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
            if len(level_candidates) > 0:
                level_cand_degrees = {k : v for k, v in degrees.items() if k in level_candidates}
                max_deg = max(list(level_cand_degrees.values()))
                max_deg_candidates = [i for i in list(level_cand_degrees.keys()) if level_cand_degrees[i] == max_deg]

                if len(max_deg_candidates) == 1:
                    v = max_deg_candidates[0]
                else:
                    v = max_deg_candidates[random.randint(0, len(max_deg_candidates) - 1)]
                clique.append(v)
                clique = DropNonAdjacentToV(clique, v, graph)
                used.append(v)
                #print("plateau", clique, level_candidates, max_deg_candidates)
        iterations -= 1
        #for i in used:
            #penalties[i] += 1
        not_clique = [i for i in vertices if i not in clique]
        u = not_clique[random.randint(0, len(not_clique) - 1)]
        clique.append(u)
        clique = DropNonAdjacentToV(clique, u, graph)
    return clique, selections#, penalties

In [25]:
def PLS_random(graph, clique, selections, max_selections, iterations, vertices, clique_size, penalties):
    used = []
    candidates = rebuild_candidates(vertices, [], graph)
    level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
    
    while iterations > 0 and selections < max_selections:
        while len(candidates) > 0 or len(level_candidates) > 0:
            candidates = rebuild_candidates(vertices, clique, graph)
            while len(candidates) > 0:
                candidates = rebuild_candidates(vertices, clique, graph)
                if len(candidates) == 1:
                    v = candidates[0]
                else:
                    v = candidates[random.randint(0, len(candidates) - 1)]
                clique.append(v)
                candidates = rebuild_candidates(vertices, clique, graph)

            if len(clique) == clique_size:
                return clique, selections#, penalties
            selections += 1
            #candidates = rebuild_candidates(vertices, [], graph)
            level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
            if len(level_candidates) > 0:
                if len(level_candidates) == 1:
                    v = level_candidates[0]
                else:
                    v = level_candidates[random.randint(0, len(level_candidates) - 1)]
                clique.append(v)
                clique = DropNonAdjacentToV(clique, v, graph)
                used.append(v)
        iterations -= 1
        #for i in used:
        #    penalties[i] += 1
        not_clique = [i for i in vertices if i not in clique]
        u = not_clique[random.randint(0, len(not_clique) - 1)]
        clique.append(u)
        clique = DropNonAdjacentToV(clique, u, graph)
    return clique, selections#, penalties

In [26]:
def PLS(graph, degrees, max_selections, clique_size):
    selections = 0
    vertices = list(i for i in range(len(graph)))
    candidates = rebuild_candidates(vertices, [], graph)
    penalties = [0] * len(vertices)
    clique = [random.randint(0, len(vertices) - 1)]

    while selections < max_selections:
        clique, selections = PLS_random(graph, clique, selections, max_selections, 50, vertices, clique_size, penalties)
        if len(clique) == clique_size:
            #print("random")
            return clique
        
        clique, selections = PLS_penalties(graph, clique, selections, max_selections, 50, vertices, clique_size, penalties)
        if len(clique) == clique_size:
            #print("penalty")
            return clique
        
        clique, selections = PLS_degrees(graph, clique, degrees, selections, max_selections, 100, vertices, clique_size, penalties)
        if len(clique) == clique_size:
            #print("degrees")
            return clique

    return None

In [27]:
def test_PLS(graph, graph_params, degrees, adj_list, max_iterations):    
    start_time = time.time()
    end_time = start_time + 50
    clique = []
    for i in range(int(graph_params[-2]), int(graph_params[-1])):
        sampe = PLS(graph, degrees, max_iterations, i)
        #AMTS(degrees, graph, adj_list, clique_size, search_depth, max_iters, graph_params)
        #print(sampe)
        if sampe == None:
            i -= 1
            break
        else:
            clique = sampe
            end_time = time.time()
            #print(clique, "\n")
            #for vertex in clique:
                #print(vertex, adj_list[vertex])
    return len(clique), end_time - start_time

CLS

In [28]:
def CLS_Penalty(graph, degrees, adj_list, max_iterations, clique_size, vertices, penalties):
    clique = [random.randint(0, len(graph) - 1)]
    penalties = [0] * len(vertices)
    used = []
    iterations = 0
    candidates = rebuild_candidates(vertices, clique, graph)
    level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
    while iterations < max_iterations:
        while len(candidates) > 0 or len(level_candidates) > 0:
            while len(candidates) > 0: 
                candidate_penalties = {k : penalties[k] for k in candidates}
                min_penalties = min(list(candidate_penalties.values()))
                min_pen_candidates = [i for i in candidates if penalties[i] == min_penalties]
                if len(min_pen_candidates) == 1:
                    v = min_pen_candidates[0]
                else:
                    v = min_pen_candidates[random.randint(0, len(min_pen_candidates) - 1)]
                clique.append(v)
                if len(clique) == clique_size:
                    return True, clique
                candidates = rebuild_candidates(vertices, clique, graph)
                level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)

            if len(level_candidates) > 0:
                candidate_penalties = {k : penalties[k] for k in level_candidates}
                min_penalties = min(list(candidate_penalties.values()))
                min_pen_candidates = [i for i in level_candidates if penalties[i] == min_penalties]
                if len(min_pen_candidates) == 1:
                    v = min_pen_candidates[0]
                else:
                    v = min_pen_candidates[random.randint(0, len(min_pen_candidates) - 1)]
                clique.append(v)
                clique = DropNonAdjacentToV(clique, v, graph)
                used.append(v)
                level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
                candidates = rebuild_candidates(vertices, clique, graph)
        iterations += 1    
        for i in range(len(penalties)):
            if penalties[i] > 0:
                penalties[i] -= 1
            if i in used:
                penalties[i] += 2

        not_clique = [i for i in vertices if i not in clique]
        u = not_clique[random.randint(0, len(not_clique) - 1)]
        clique.append(u)
        clique = DropNonAdjacentToV(clique, u, graph)       
        level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
        candidates = rebuild_candidates(vertices, clique, graph)
    return False, clique

In [29]:
def CLS_Focus(graph, degrees, adj_list, max_iterations, clique_size, vertices, aim_degree):
    clique = [random.randint(0, len(graph) - 1)]
    used = []
    iterations = 0
    candidates = rebuild_candidates(vertices, clique, graph)
    level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
    while iterations < max_iterations:
        while len(candidates) > 0 or len(level_candidates) > 0:
            while len(candidates) > 0: 
                cand_degrees = {k : abs(v - aim_degree) for k, v in degrees.items() if k in candidates}
                goal = min(list(cand_degrees.values()))
                goal_deg_candidates = [i for i in list(cand_degrees.keys()) if cand_degrees[i] == goal]
                if len(goal_deg_candidates) == 1:
                    v = goal_deg_candidates[0]
                else:
                    v = goal_deg_candidates[random.randint(0, len(goal_deg_candidates) - 1)]
                clique.append(v)
                if len(clique) == clique_size:
                    return clique
                candidates = rebuild_candidates(vertices, clique, graph)
                level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)

            if len(level_candidates) > 0:
                level_cand_degrees = {k : abs(v - aim_degree) for k, v in degrees.items() if k in level_candidates}
                goal = min(list(level_cand_degrees.values()))
                goal_deg_candidates = [i for i in list(level_cand_degrees.keys()) if level_cand_degrees[i] == goal]
                if len(goal_deg_candidates) == 1:
                    v = goal_deg_candidates[0]
                else:
                    v = goal_deg_candidates[random.randint(0, len(goal_deg_candidates) - 1)]
                clique.append(v)
                clique = DropNonAdjacentToV(clique, v, graph)
                used.append(v)
                level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
                candidates = rebuild_candidates(vertices, clique, graph)
            #print(iterations)
        iterations += 1    

        not_clique = [i for i in vertices if i not in clique]
        u = not_clique[random.randint(0, len(not_clique) - 1)]
        clique.append(u)
        clique = DropNonAdjacentToV(clique, u, graph) 
        level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
        candidates = rebuild_candidates(vertices, clique, graph)      
    return None

In [30]:
def CLS_Level(graph, degrees, adj_list, max_iterations, clique_size, vertices, penalties):
    clique = [random.randint(0, len(graph) - 1)]
    #penalties = [0] * len(vertices)
    used = []
    iterations = 0
    candidates = rebuild_candidates(vertices, clique, graph)
    level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
    while iterations < max_iterations:
        while len(candidates) > 0 or len(level_candidates) > 0:
            while len(candidates) > 0: 
                #print("Add", iterations)
                cand_degrees = {k : v for k, v in degrees.items() if k in candidates}
                max_deg = max(list(cand_degrees.values()))
                max_deg_candidates = [i for i in list(cand_degrees.keys()) if cand_degrees[i] == max_deg]
                if len(max_deg_candidates) == 1:
                    v = max_deg_candidates[0]
                else:
                    v = max_deg_candidates[random.randint(0, len(max_deg_candidates) - 1)]
                clique.append(v)
                if len(clique) == clique_size:
                    return True, clique, penalties
                candidates = rebuild_candidates(vertices, clique, graph)
                level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)

            if len(level_candidates) > 0:
                #print("Change", iterations)
                level_cand_degrees = {k : v for k, v in degrees.items() if k in level_candidates}
                max_deg = max(list(level_cand_degrees.values()))
                max_deg_candidates = [i for i in list(level_cand_degrees.keys()) if level_cand_degrees[i] == max_deg]
                if len(max_deg_candidates) == 1:
                    v = max_deg_candidates[0]
                else:
                    v = max_deg_candidates[random.randint(0, len(max_deg_candidates) - 1)]
                clique.append(v)
                clique = DropNonAdjacentToV(clique, v, graph)
                used.append(v)
                level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
                candidates = rebuild_candidates(vertices, clique, graph)

        iterations += 1   
        #print("Drop", iterations)
        v_to_drop = clique[random.randint(0, len(clique) - 1)]
        clique = [i for i in clique if i != v_to_drop]
        level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
        candidates = rebuild_candidates(vertices, clique, graph)
        if len(candidates) == 0 and len(level_candidates) == 0:
            not_clique = [i for i in vertices if i not in clique]
            u = not_clique[random.randint(0, len(not_clique) - 1)]
            clique.append(u)
            clique = DropNonAdjacentToV(clique, u, graph)
            level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
            candidates = rebuild_candidates(vertices, clique, graph)

        for i in range(len(penalties)):
            if i in used:
                penalties[i] += 1
       
    return False, clique, penalties

In [31]:
def CLS_Greedy(graph, degrees, adj_list, max_iterations, clique_size, vertices):
    clique = [random.randint(0, len(graph) - 1)]
    penalties = [0] * len(vertices)
    used = []
    iterations = 0
    candidates = rebuild_candidates(vertices, clique, graph)
    level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
    while iterations < max_iterations:
        while len(candidates) > 0 or len(level_candidates) > 0:
            while len(candidates) > 0: 
                cand_degrees = {k : v for k, v in degrees.items() if k in candidates}
                max_deg = max(list(cand_degrees.values()))
                max_deg_candidates = [i for i in list(cand_degrees.keys()) if cand_degrees[i] == max_deg]
                if len(max_deg_candidates) == 1:
                    v = max_deg_candidates[0]
                else:
                    v = max_deg_candidates[random.randint(0, len(max_deg_candidates) - 1)]
                clique.append(v)
                if len(clique) == clique_size:
                    return True, clique, penalties
                candidates = rebuild_candidates(vertices, clique, graph)
                level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)

            if len(level_candidates) > 0:
                level_cand_degrees = {k : v for k, v in degrees.items() if k in level_candidates}
                max_deg = max(list(level_cand_degrees.values()))
                max_deg_candidates = [i for i in list(level_cand_degrees.keys()) if level_cand_degrees[i] == max_deg]
                if len(max_deg_candidates) == 1:
                    v = max_deg_candidates[0]
                else:
                    v = max_deg_candidates[random.randint(0, len(max_deg_candidates) - 1)]
                clique.append(v)
                clique = DropNonAdjacentToV(clique, v, graph)
                used.append(v)
                level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
                candidates = rebuild_candidates(vertices, clique, graph)
        iterations += 1    
        #print(iterations)
        for i in range(len(penalties)):
            if i in used:
                penalties[i] += 1

        not_clique = [i for i in vertices if i not in clique]
        u = not_clique[random.randint(0, len(not_clique) - 1)]
        clique.append(u)
        clique = DropNonAdjacentToV(clique, u, graph)   
        level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
        candidates = rebuild_candidates(vertices, clique, graph)
 
    return False, clique, penalties

In [32]:
def CLS(graph, degrees, adj_list, max_iterations, clique_size):
    vertices = [i for i in range(len(graph))]
    #print("Greedy")
    success, clique, penalties = CLS_Greedy(graph, degrees, adj_list, max_iterations, clique_size, vertices)
    if success:
        return clique
    else:
        #print("Level", clique)
        success, clique, penalties = CLS_Level(graph, degrees, adj_list, max_iterations, clique_size, vertices, penalties)
        if success:
            return clique
        else:
            #print("Penalty", clique)
            success, clique = CLS_Penalty(graph, degrees, adj_list, max_iterations, clique_size, vertices, penalties)
            if success:
                return clique
            else:
                aim_degree = 0
                for vertex in clique:
                    aim_degree += degrees[vertex]
                aim_degree // len(clique)
                #print("Focus", aim_degree)
                clique = CLS_Focus(graph, degrees, adj_list, max_iterations, clique_size, vertices, aim_degree)

    return clique

In [33]:
def test_CLS(graph, graph_params, degrees, adj_list, max_iterations):
    start_time = time.time()
    end_time = start_time + 50
    clique = []
    for i in range(int(graph_params[-2]), int(graph_params[-1])):
        sampe = CLS(graph, degrees, adj_list, max_iterations, i)
        #print(sampe)
        if sampe == None:
            i -= 1
            break
        else:
            clique = sampe
            end_time = time.time()
            #print(clique, "\n")
            #for vertex in clique:
                #print(vertex, adj_list[vertex])
    return len(clique), end_time - start_time

SBTS

In [34]:
def random_start(graph, vertices):
    clique = []
    candidates = rebuild_candidates(vertices, clique, graph)
    while len(candidates) > 1:
        clique.append(candidates[random.randint(0, len(candidates) - 1)])
        candidates = rebuild_candidates(candidates, clique, graph)
    if len(candidates) == 1:
        clique.append(candidates[0])
    return clique

In [35]:
def mapping_degrees(subset, clique, adj_list):
    missing = {}
    for vertex in subset:
        missing_subclique = []
        for elem in clique:
            if elem not in adj_list[vertex]:
                missing_subclique.append(elem)
        missing[vertex] = [missing_subclique, len(missing_subclique)]
    return missing         

In [36]:
def expanding_degrees(subset, not_clique, adj_list, mapping_degrees):
    degrees = {}
    for vertex in subset:
        counter = 0
        for elem in adj_list[vertex]:
            if elem in not_clique and mapping_degrees[elem][1] == 1:
                counter += 1
        degrees[vertex] = counter
    return degrees

In [37]:
def diversifying_degrees(graph, clique, not_clique):
    degrees = {}
    for vertex_1 in not_clique:
        counter = 0
        for vertex_2 in clique:
            if graph[vertex_1][vertex_2] > 0:
                counter += 1
        degrees[vertex_1] = counter
    return degrees

In [38]:
def update_level_candidates_SBTS(level_candidates, clique, current_expanding_degrees, graph):
    result = []
    for candidate in level_candidates:
        drop = False
        for vertex in clique:
            if graph[candidate][vertex] > 0 and current_expanding_degrees[vertex] == 1:
                drop = True
                break
        if drop == False:
            result.append(candidate)
    return result

In [39]:
def SBTS_Intensify(graph, adj_list, clique, candidates, level_candidates, vertices, taboo_list):
    #print("Intensify")
    used = []
    dropped = []
    candidates = [i for i in rebuild_candidates(vertices, clique, graph) if taboo_list[i] == 0]
    #print("candidates", candidates)
    while len(candidates) > 0:
        if len(candidates) == 1:
            v = candidates[0]
        else:
            v = candidates[random.randint(0, len(candidates) - 1)]
        clique.append(v)
        candidates = [i for i in rebuild_candidates(vertices, clique, graph) if taboo_list[i] == 0]
        level_candidates = [i for i in rebuild_level_neighbourhood(clique, graph, vertices, used) if taboo_list[i] == 0]
    #print("candidates", candidates)
    #print("level", level_candidates)
    if len(level_candidates) > 0:
        not_clique = [i for i in vertices if i not in clique]
        current_mapping_degrees = mapping_degrees(not_clique, clique, adj_list)
        two_plus_candidates = [i for i in list(current_mapping_degrees.keys()) if i not in level_candidates]
        current_expanding_degrees = expanding_degrees(vertices, not_clique, adj_list, current_mapping_degrees)
        #print("two plus", two_plus_candidates)
        if len(level_candidates) > len(two_plus_candidates):
            level_candidates = update_level_candidates_SBTS(level_candidates, clique, current_expanding_degrees, graph)
            #print("changing candidates and degrees", level_candidates, current_expanding_degrees)
        current_expanding_degrees = {k : v for k, v in current_expanding_degrees.items() if k in level_candidates}
        max_exp_degree = max(list(current_expanding_degrees.values()))
        level_max_exp = [i for i in level_candidates if current_expanding_degrees[i] == max_exp_degree]
        #print("max exp", max_exp_degree, level_max_exp)
        if len(level_max_exp) == 0:
            clique = clique
        else:
            if len(level_max_exp) == 1:
                v = level_max_exp[0]
            else:
                current_diversifying_degrees = diversifying_degrees(graph, clique, not_clique)
                #print(current_diversifying_degrees)
                current_diversifying_degrees = {k : v for k, v in current_diversifying_degrees.items() if k in level_max_exp}
                #print("div degree", current_diversifying_degrees, level_max_exp)
                max_div_degree = max(list(current_diversifying_degrees.values()))
                level_max_div = [i for i in level_max_exp if current_diversifying_degrees[i] == max_div_degree]
                if len(level_max_div) == 1:
                    v = level_max_div[0]
                else:
                    v = level_max_div[random.randint(0, len(level_max_div) - 1)]
            old_clique = clique
            #print("change", v)
            clique.append(v)
            used.append(v)
            clique = DropNonAdjacentToV(clique, v, graph)
            dropped = [i for i in old_clique if i not in clique]
        candidates = [i for i in rebuild_candidates(vertices, clique, graph) if i not in taboo_list]
        level_candidates = [i for i in rebuild_level_neighbourhood(clique, graph, vertices, used) if taboo_list[i] == 0]
    return clique, candidates, level_candidates, dropped

In [40]:
def SBTS_diersify(graph, adj_list, clique, vertices, taboo_list):
    #print("clique", clique)
    dropped = []
    not_clique = [i for i in range(len(graph)) if i not in clique]
    level_candidates = [i for i in rebuild_level_neighbourhood(clique, graph, vertices, []) if taboo_list[i] == 0]
    not_clique = [i for i in vertices if i not in clique]
    current_mapping_degrees = mapping_degrees(not_clique, clique, adj_list)
    two_plus_candidates = [i for i in list(current_mapping_degrees.keys()) if i not in level_candidates and taboo_list[i] == 0]

    # print("Taboo", taboo_list)
    # alt_level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, [])
    # alt_two_plus_candidates = [i for i in list(current_mapping_degrees.keys()) if i not in level_candidates and taboo_list[i] == 0]
    # print(alt_level_candidates, alt_two_plus_candidates)
    # print(level_candidates, two_plus_candidates)
    three_plus_candidates = [i for i in list(current_mapping_degrees.keys()) if current_mapping_degrees[i][1] > 2 and taboo_list[i] == 0]

    if len(level_candidates) > len(two_plus_candidates) and len(three_plus_candidates) > 0:
        current_diversifying_degrees = diversifying_degrees(graph, clique, not_clique)
        current_diversifying_degrees = {k : v for k, v in current_diversifying_degrees.items() if k in three_plus_candidates}
        #print(level_candidates, three_plus_candidates, current_diversifying_degrees)
        max_div_degree = max(list(current_diversifying_degrees.values()))
        level_max_div = [i for i in three_plus_candidates if current_diversifying_degrees[i] == max_div_degree]
        if len(level_max_div) == 1:
            v = level_max_div[0]
        else:
            v = level_max_div[random.randint(0, len(level_max_div) - 1)]
    else:
        #print("two plus", two_plus_candidates)
        #print("level", level_candidates)
        if random.randint(0, 1) == 1 and len(two_plus_candidates) > 0:
            two_candidates = [i for i in list(current_mapping_degrees.keys()) if current_mapping_degrees[i][1] == 2 and taboo_list[i] == 0]
            if len(two_candidates) > 0:
                current_diversifying_degrees = diversifying_degrees(graph, clique, not_clique)
                current_diversifying_degrees = {k : v for k, v in current_diversifying_degrees.items() if k in two_candidates}
                #print("two plus, two, current mapping degrees div degrees", two_plus_candidates, two_candidates, current_mapping_degrees, current_diversifying_degrees)
                max_div_degree = max(list(current_diversifying_degrees.values()))
                level_max_div = [i for i in two_candidates if current_diversifying_degrees[i] == max_div_degree]
                if len(level_max_div) == 1:
                    v = level_max_div[0]
                else:
                    v = level_max_div[random.randint(0, len(level_max_div) - 1)]
            else:
                two_candidates = [i for i in list(current_mapping_degrees.keys()) if current_mapping_degrees[i][1] == 2]
                if len(two_candidates) > 0:
                    current_diversifying_degrees = diversifying_degrees(graph, clique, not_clique)
                    current_diversifying_degrees = {k : v for k, v in current_diversifying_degrees.items() if k in two_candidates}
                    #print("two plus, two, current mapping degrees div degrees", two_plus_candidates, two_candidates, current_mapping_degrees, current_diversifying_degrees)
                    max_div_degree = max(list(current_diversifying_degrees.values()))
                    level_max_div = [i for i in two_candidates if current_diversifying_degrees[i] == max_div_degree]
                    if len(level_max_div) == 1:
                        v = level_max_div[0]
                    else:
                        v = level_max_div[random.randint(0, len(level_max_div) - 1)]
        else:
            if len(two_plus_candidates) == 1:
                v = two_plus_candidates[0]
            else:
                if len(two_plus_candidates) > 0:
                    v = two_plus_candidates[random.randint(0, len(two_plus_candidates) - 1)]
                else:
                    return clique, dropped
    old_clique = clique
    clique.append(v)
    clique = DropNonAdjacentToV(clique, v, graph)
    dropped = [i for i in old_clique if i not in clique]
    return clique, dropped

In [41]:
def update_taboo_list(taboo_list, graph, vertices, clique, adj_list, moved):
    if len(moved) == 1:
        used = []
        not_clique = [i for i in range(len(graph)) if i not in clique]
        level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, used)
        not_clique = [i for i in vertices if i not in clique]
        current_mapping_degrees = mapping_degrees(not_clique, clique, adj_list)
        two_plus_candidates = [i for i in list(current_mapping_degrees.keys()) if i not in level_candidates]
        if len(level_candidates) > len(two_plus_candidates):
            taboo_list[moved[0]] = 10 + random.randint(0, len(level_candidates) - 1)
        else:
            taboo_list[moved[0]] = len(level_candidates)
    else:
        if len(moved) == 0:
            for i in range(len(taboo_list)):
                if taboo_list[i] > 0:
                    taboo_list[i] -= 1
        else:
            for i in moved:
                taboo_list[i] = 7
    
    return taboo_list

In [42]:
def SBTS(graph, degrees, adj_list, max_iterations):
    start = time.time()
    vertices = [i for i in range(len(graph))]
    clique = random_start(graph, vertices)
    best_clique = clique
    end = time.time()
    best_clique_size = len(clique)
    taboo_list = [0] * len(graph)
    iterations = 0
    dropped = []
    candidates = rebuild_candidates(vertices, clique, graph)
    level_candidates = rebuild_level_neighbourhood(clique, graph, vertices, [])

    while iterations < max_iterations:
        #print(candidates, level_candidates)
        #print(iterations, clique)
        if len(candidates) > 0 or len(level_candidates) > 0:
            clique, candidates, level_candidates, dropped = SBTS_Intensify(graph, adj_list, clique, candidates, level_candidates, vertices, taboo_list)
            if len(clique) > best_clique_size:
                end = time.time()
                best_clique = clique
                best_clique_size = len(clique)
            update_taboo_list(taboo_list, graph, vertices, clique, adj_list, dropped)
        else:
            if len(clique) > 1:
                clique, dropped = SBTS_diersify(graph, adj_list, clique, vertices, taboo_list)
            else:
                not_clique = [i for i in range(len(graph)) if i not in clique]
                v = not_clique[random.randint(0, len(not_clique) - 1)]
                dropped = clique
                clique.append(v)
                clique = DropNonAdjacentToV(clique, v, graph)
            taboo_list = update_taboo_list(taboo_list, graph, vertices, clique, adj_list, dropped)
        
        update_taboo_list(taboo_list, graph, vertices, clique, adj_list, [])
        iterations += 1

    return best_clique, len(best_clique), end - start

In [43]:
def test_SBTS(graph, degrees, adj_list, max_iterations):
    #start = time.time()
    clique, clique_len, exec_time = SBTS(graph, degrees, adj_list, max_iterations)
    #end = time.time()
    return clique_len, exec_time

ТЕСТЫ

In [44]:
def test_all(graph, graph_params, degrees, adj_list, max_iters, n_times):
    results = {}
    best_clique_size, sum_time = 0, 0
    for i in range(n_times):
        clique_size, current_time = test_AMTS(graph, graph_params, degrees, adj_list, max_iters)
        if clique_size > best_clique_size:
            best_clique_size = clique_size
        sum_time += current_time
    results[0] = [best_clique_size, sum_time]

    best_clique_size, sum_time = 0, 0
    for i in range(n_times):
        clique_size, current_time = test_DLS(graph, graph_params, degrees, adj_list, max_iters)
        if clique_size > best_clique_size:
            best_clique_size = clique_size
        sum_time += current_time
    results[1] = [best_clique_size, sum_time]

    best_clique_size, sum_time = 0, 0
    for i in range(n_times):
        clique_size, current_time = test_PLS(graph, graph_params, degrees, adj_list, max_iters)
        if clique_size > best_clique_size:
            best_clique_size = clique_size
        sum_time += current_time
    results[2] = [best_clique_size, sum_time]

    best_clique_size, sum_time = 0, 0
    for i in range(n_times):
        clique_size, current_time = test_CLS(graph, graph_params, degrees, adj_list, max_iters)
        if clique_size > best_clique_size:
            best_clique_size = clique_size
        sum_time += current_time
    results[3] = [best_clique_size, sum_time]

    best_clique_size, sum_time = 0, 0
    for i in range(n_times):
        clique_size, current_time = test_SBTS(graph, degrees, adj_list, max_iters)
        if clique_size > best_clique_size:
            best_clique_size = clique_size
        sum_time += current_time
    results[4] = [best_clique_size, sum_time]

    return results

In [45]:
def find_best_algorithm(result): 
    #result = {0: [21, 16.93840217590332], 1: [22, 0.9338009357452393], 2: [22, 0.33661413192749023], 3: [21, 0.22690391540527344], 4: [21, 11.393150568008423]}
    clique_sizes = []
    for key in list(result.keys()):
        clique_sizes.append(result[key][0])
    max_clique_size = max(clique_sizes)
    best_solutions = [i for i in list(result.keys()) if result[i][0] == max_clique_size]
    min_time = 100000
    best_algorithm = -1
    for solution in best_solutions:
        if result[solution][1] < min_time:
            min_time = result[solution][1]
            best_algorithm = solution
    return best_algorithm

In [46]:
def execute_on_graphs_series(filename_schematic, max_iterations,n_times):    
    best = []
    for k in range(10):
        filename = filename_schematic + str(k) + ".csv"
        graph = np.genfromtxt(filename, delimiter = " ,")
        parms_filename = "paramsI" + filename
        graph_params = np.genfromtxt(parms_filename, delimiter = " ,")
        graph, degrees, adj_list = get_graph_lists(graph)
        results = test_all(graph, graph_params, degrees, adj_list, max_iterations, n_times)
        best_algorithm = find_best_algorithm(results)
        best.append(best_algorithm)

    results_filename = "resultsI" + filename
    print(best, results_filename)
    best = np.array(best)
    np.savetxt(results_filename, best, delimiter = " ,")
    return 

In [49]:
filename = "graphI500I"
for i in range(60, 65, 5):
    execute_on_graphs_series(filename + str(i) + "I", 250, 5)

[3, 1, 2, 1, 1, 3, 2, 1, 1, 3] resultsIgraphI500I60I9.csv


In [ ]:
filename = "graphI150I95I1.csv"
graph = np.genfromtxt(filename, delimiter = " ,")

graph_params = np.genfromtxt("paramsI" + filename, delimiter = " ,")
graph, degrees, adj_list = get_graph_lists(graph)

In [ ]:
results = test_all(graph, graph_params, degrees, adj_list, 500, 5)
print(results, find_best_algorithm(results))

{0: [21, 52.904942750930786], 1: [22, 3.300570487976074], 2: [22, 2.7230067253112793], 3: [22, 6.838778972625732], 4: [23, 7.916337966918945]} 2


In [ ]:
print("AMTS", test_AMTS(graph, graph_params, degrees, adj_list, 750))
print("DLS", test_DLS(graph, graph_params, degrees, adj_list, 750))
print("PLS", test_PLS(graph, graph_params, degrees, adj_list, 750))
print("CLS", test_CLS(graph, graph_params, degrees, adj_list, 750))
print("SBTS", test_SBTS(graph, degrees, adj_list, 750))

AMTS (50, 20.40083885192871)
DLS (52, 7.745387077331543)
PLS (51, 7.373023986816406)
CLS (51, 5.665826082229614)
SBTS (52, 10.713024854660034)


СРАВНЕНИЕ НЕЙРОННЫХ МОДЕЛЕЙ

In [52]:
import numpy as np

from scipy.sparse import coo_matrix
import fast_matrix_market as fmm
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

params = [[150, 0.95], [200, 0.95], [300, 0.85], [400, 0.6], [500, 0.6], [1000, 0.4], [1500, 0.4]]

In [68]:
def get_graph_lists_obj(graph): # Преобразование графа в список и получение списков смежности и степеней для вершин
    graph = list(graph)
    for i in range(len(graph)):
        graph[i] = list(graph[i])

    adj_list = {}
    degrees = {}
    for i in range(len(graph)):
        temp = []
        degree = 0
        for j in range(len(graph)):
            if graph[i][j] == 1.0:
                degree += 1
                temp.append(j)
        adj_list[i] = temp
        degrees[i] = degree

    degrees = {k: v for k, v in sorted(degrees.items(), key=lambda item: item[1], reverse = True)}
    return graph, degrees, adj_list

In [75]:
def greedy_search(degrees, graph): #Жадный поиск
    candidates = list(degrees.keys())
    clique = [candidates[0]]

    while True:
        candidate_index = 0 
        to_drop = []

        #print(clique, candidates)

        for candidate in candidates: #Перестроение списка кандидатов
            for vertex in clique:
                if graph[vertex][candidate] == 0:
                    to_drop.append(candidate_index)
                    break
                
            candidate_index += 1

        candidates = [i for j, i in enumerate(candidates) if j not in to_drop]
        
        if len(candidates) == 0: #Если кандидаты ещё есть - добавление в клику лучшего, иначе завершение программы
            break
        clique.append(candidates[0])

    return clique

In [56]:
def greedy_coloring(adj_list): #Жадное окрашивание
    vertices = sorted(list(adj_list.keys()))
    colour_graph = {}

    for vertex in vertices:
        unused_colours = len(vertices) * [True] #Список неиспользованных цветов

        for neighbor in adj_list[vertex]: #Отмечаются все занятые соседями цвета
            if neighbor in colour_graph:
                colour = colour_graph[neighbor] 
                unused_colours[colour] = False

        for colour, unused in enumerate(unused_colours): #Вершине присваивается первый незанятый цвет
            if unused:
                colour_graph[vertex] = colour
                break

    return colour_graph

In [76]:
def get_graph_params(graph):
    graph, degrees, adj_list = get_graph_lists_obj(graph)
    greedy_clique = greedy_search(degrees, graph)
    colored = greedy_coloring(adj_list)

    gs_res = len(greedy_clique)

    gc_res = len(np.unique(list(colored.values())))

    vertex_num = len(graph)

    edges_num = 0
    degrees_sum = []
    for i in range(vertex_num):
        for j in range(i, vertex_num):
            if graph[i][j] > 0:
                edges_num += 1
                degrees_sum.append(degrees[i] + degrees[j])
    density = edges_num / vertex_num

    degrees_list = list(degrees.values())
    min_deg = min(degrees_list)
    max_deg = max(degrees_list)
    mean_deg = np.mean(degrees_list)
    std_deg = np.std(degrees_list)

    deg_sum_max = max(degrees_sum)
    deg_sum_min = min(degrees_sum)
    deg_sum_mean = np.mean(degrees_sum)
    deg_sum_std = np.std(degrees_sum)
    
    graph_params = []
    graph_params.append(vertex_num)
    graph_params.append(edges_num)
    graph_params.append(density)
    graph_params.append(min_deg)
    graph_params.append(max_deg)
    graph_params.append(mean_deg)
    graph_params.append(std_deg)
    graph_params.append(deg_sum_min)
    graph_params.append(deg_sum_max)
    graph_params.append(deg_sum_mean)
    graph_params.append(deg_sum_std)
    graph_params.append(gs_res)
    graph_params.append(gc_res)

    return graph_params

In [58]:
features = []
answers = []

for i in range(len(params)):
    limit = int(params[i][1] // 0.05)
    for j in range(limit + 1):
        possibility = min((j + 1) * 0.05, params[i][1])
        filename = "graphI" + str(params[i][0]) + "I" + str(int(possibility * 100)) + "I"
        serie_results = list(np.genfromtxt("resultsI" + filename + "9.csv", delimiter = " ,"))
        answers += serie_results
        for k in range(10):
            graph_params = list(np.genfromtxt("paramsI" + filename + str(k) + ".csv", delimiter = " ,"))
            features.append(graph_params)

In [59]:
log_regr = LogisticRegression(solver='lbfgs', max_iter=10000, multi_class='ovr')
log_regr.fit(features, answers)
lin_regr = LinearRegression()
lin_regr.fit(features, answers)
svc = svm.SVC()
svc.fit(features, answers)
bayes = GaussianNB()
bayes.fit(features, answers)
trees = tree.DecisionTreeClassifier()
trees.fit(features, answers)

DecisionTreeClassifier()

In [62]:
def pred_on_dimacs_graph(filename):
    (data, (rows, cols)), shape = fmm.read_coo(filename)
    graph = coo_matrix((data, (rows, cols)), shape).toarray()
    graph_params = get_graph_params(graph)
    log_res = int(log_regr.predict(np.array(graph_params).reshape(1, -1)))
    lin_res = int(lin_regr.predict(np.array(graph_params).reshape(1, -1)))
    svc_res = int(svc.predict(np.array(graph_params).reshape(1, -1)))
    bayes_res = int(bayes.predict(np.array(graph_params).reshape(1, -1)))
    trees_res = int(trees.predict(np.array(graph_params).reshape(1, -1)))

    return {"logical" : log_res, "linear": lin_res, "vector" : svc_res, "bayes" : bayes_res, "tree" : trees_res}

In [83]:
def test_all_on_dimacs(filename):    
    (data, (rows, cols)), shape = fmm.read_coo(filename)
    graph = coo_matrix((data, (rows, cols)), shape).toarray()
    graph_params = get_graph_params(graph)
    graph, degrees, adj_list = get_graph_lists_obj(graph)
    results = test_all(graph, graph_params, degrees, adj_list, 500, 5)
    print(results, find_best_algorithm(results))
    return results, find_best_algorithm(results)

In [78]:
def compare_classificators(filename):
    results = pred_on_dimacs_graph(filename)
    print(results)
    test_all_on_dimacs(filename)
    return

In [91]:
dimacs = ["brock200-2.mtx", "brock200-4.mtx", "brock400-1.mtx", "brock400-3.mtx", "brock800-1.mtx", "brock800-3.mtx"]

In [92]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
for graph in dimacs:
    print("\n")
    compare_classificators(graph)



{'logical': 4, 'linear': 3, 'vector': 4, 'bayes': 4, 'tree': 4}
{0: [9, 7.085291862487793], 1: [11, 2.380017042160034], 2: [11, 1.355820655822754], 3: [12, 34.45299768447876], 4: [11, 2.3343305587768555]} 3


{'logical': 4, 'linear': 3, 'vector': 4, 'bayes': 4, 'tree': 1}
{0: [14, 29.793453216552734], 1: [16, 2.1580920219421387], 2: [16, 6.24228048324585], 3: [16, 5.718635559082031], 4: [16, 1.7407643795013428]} 4


{'logical': 4, 'linear': 3, 'vector': 4, 'bayes': 1, 'tree': 3}
{0: [21, 169.12297177314758], 1: [23, 12.571671962738037], 2: [23, 12.900123596191406], 3: [24, 188.0652039051056], 4: [24, 90.73415350914001]} 4


{'logical': 4, 'linear': 3, 'vector': 4, 'bayes': 1, 'tree': 3}
{0: [21, 167.46181416511536], 1: [23, 12.697833776473999], 2: [24, 28.29514789581299], 3: [24, 303.1239285469055], 4: [23, 24.436346292495728]} 2


{'logical': 3, 'linear': 3, 'vector': 4, 'bayes': 3, 'tree': 4}
{0: [18, 327.744056224823], 1: [19, 28.846678495407104], 2: [20, 66.08156681060791], 3: [2

In [93]:
dimacs_2 = ["brock200-1.mtx", "brock200-3.mtx", "brock400-2.mtx", "brock400-4.mtx", "brock800-2.mtx", "brock800-4.mtx"]
for graph in dimacs_2:
    print("\n")
    compare_classificators(graph)



{'logical': 4, 'linear': 3, 'vector': 4, 'bayes': 4, 'tree': 4}
{0: [19, 74.1595025062561], 1: [20, 4.575829029083252], 2: [21, 7.86239767074585], 3: [20, 17.202847242355347], 4: [20, 5.908117294311523]} 2


{'logical': 4, 'linear': 3, 'vector': 4, 'bayes': 4, 'tree': 1}
{0: [13, 18.409181594848633], 1: [13, 1.393322467803955], 2: [14, 3.280411720275879], 3: [14, 22.94440984725952], 4: [15, 1.3860337734222412]} 4


{'logical': 4, 'linear': 2, 'vector': 4, 'bayes': 1, 'tree': 3}
{0: [21, 229.0097155570984], 1: [23, 20.570472955703735], 2: [24, 30.63174295425415], 3: [24, 107.91820025444031], 4: [24, 36.56958556175232]} 2


{'logical': 2, 'linear': 3, 'vector': 4, 'bayes': 1, 'tree': 3}
{0: [20, 114.2393217086792], 1: [23, 23.644347429275513], 2: [24, 29.020169734954834], 3: [24, 327.62545800209045], 4: [25, 100.87111496925354]} 4


{'logical': 3, 'linear': 2, 'vector': 4, 'bayes': 3, 'tree': 4}
{0: [18, 701.0199453830719], 1: [20, 40.06808161735535], 2: [19, 44.73257899284363], 3: [21